# Dept for Culture, Media & Sport 

## Total Income of DCMS-funded cultural organisations 2022/2023

In [63]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import re

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:,.0f}'.format)

In [64]:
DATA_DIR = Path('../../data/dcms/Total_income_for_DCMS-funded_cultural_organisations_2022_23_data_tables_Final_SL.ods')
OUT_DIR = Path('../../src/data/dcms/funding/_data')

In [65]:
def process_sheet(data): 
    data = data.replace('x', 0).dropna(axis='columns', how='all')
    data_latest = data[['Name of organisation', '2022/2023']]

    data_latest = (
        data_latest.loc[data_latest['2022/2023'] != 0]
        .loc[data_latest['Name of organisation'] != 'Grand Total']
    )

    data_latest['Name of organisation'] = (
        data_latest['Name of organisation']
        .apply(lambda x: re.sub(r'\[note \d+\]', '', x))
    )

    data_latest['2022/2023'] = data_latest['2022/2023'].round(0).astype(int)
    
    return data_latest

def clean_orgs(name):
    name = re.sub('Of which from:Arts Council England ', '', name)
    return re.sub(r'\[note \d+\]', '', name)

In [66]:
grant_in_aid = pd.read_excel(DATA_DIR, engine='odf', sheet_name=['3'], skiprows=4)
total_income_generated = pd.read_excel(DATA_DIR, engine='odf', sheet_name=['5'], skiprows=5)

grant_in_aid = pd.DataFrame(grant_in_aid['3'])
total_income_generated = pd.DataFrame(total_income_generated['5'])

In [79]:
grant_in_aid = grant_in_aid[['Name of organisation', '2022/2023']]
total_income_generated = total_income_generated[['Name of organisation', '2022/2023']]

grant_in_aid['Name of organisation'] = grant_in_aid['Name of organisation'].apply(lambda x: clean_orgs(x))
total_income_generated['Name of organisation'] = grant_in_aid['Name of organisation'].apply(lambda x: clean_orgs(x))


merged = (
    total_income_generated.merge(grant_in_aid, how='outer', on='Name of organisation')
    .rename(columns={
        'Name of organisation': 'organisation',
        '2022/2023_x': 'total_income',
        '2022/2023_y': 'gift_in_aid'
    })
    .replace('x', np.nan)
)
merged = merged.loc[merged['organisation']!='Grand Total']

merged = (
    merged.loc[merged['organisation']!='Grand Total']
    .set_index('organisation')
    .dropna(axis='rows', how='all')
    .fillna(0).astype(int)
)


merged = merged.reset_index()

ace_funding = merged.loc[
    merged['organisation'].isin(['National Portfolio Organisations ', 'Arts Council England'])
]

ace_funding.to_csv(Path(OUT_DIR / 'ace_funding.csv'), index=False)

merged = merged.loc[
    ~merged['organisation'].isin(['National Portfolio Organisations ', 'Arts Council England'])
]

merged.to_csv(Path(OUT_DIR/'income_vs_funding.csv'), index=False)


/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_95930/4187259311.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('x', np.nan)


### Process Grant-in-Aid figures

In [ ]:
grant_in_aid = process_sheet(grant_in_aid)
grant_in_aid['Name of organisation'] = grant_in_aid['Name of organisation'].apply(lambda x: re.sub('Of which from:Arts Council England ', '', x))

grant_in_aid.to_csv(Path(OUT_DIR/ 'grant_in_aid_latest.csv'), index=False)


/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_84711/2741843527.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('x', 0).dropna(axis='columns', how='all')


### Process total income

In [ ]:
total_income_generated = process_sheet(total_income_generated)
total_income_generated.to_csv(Path(OUT_DIR / 'total_income_generated_latest.csv'), index=False)

/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_84711/2741843527.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('x', 0).dropna(axis='columns', how='all')
